In [4]:
import os
from openai import AzureOpenAI
from IPython.display import display, Markdown
import json

openai_endpoint = os.getenv("OPENAI_ENDPOINT")
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_version = "2024-06-01"
openai_completion_model = "gpt-4o"

client = AzureOpenAI(
    azure_endpoint = openai_endpoint, 
    api_key = openai_api_key, 
    api_version = openai_version)

In [22]:
persona = """Based on the input of the user you categorize the message as NEGATIVE OR POSITIVE and also return a list of keywords, respond in the following JSON format: { "sentiment": "", "keywords": "" }"""

completion = client.chat.completions.create(
    model = openai_completion_model,
    messages = [
        {"role": "system", "content": persona},
        {"role": "user", "content": "I really dislike the way things are done in this organization?"},
    ]
)

display(Markdown(completion.choices[0].message.content))
_dict = json.loads(completion.choices[0].message.content)
_dict

{
  "sentiment": "NEGATIVE",
  "keywords": ["dislike", "organization", "things are done"]
}

{'sentiment': 'NEGATIVE',
 'keywords': ['dislike', 'organization', 'things are done']}

In [15]:
persona = """Based on the input of the user you categorize the message as NEGATIVE OR POSITIVE and also return a list of keywords, respond in the following JSON format: { "sentiment": "", "keywords": "" }"""

completion = client.chat.completions.create(
    model = openai_completion_model,
    messages = [
        {"role": "system", "content": persona},
        {"role": "user", "content": "I really dislike the way things are done in this organization?"},
    ]
)

display(Markdown(completion.choices[0].message.content))
#_dict = json.loads(completion.choices[0].message.content)
#_dict


```json
{
  "sentiment": "NEGATIVE",
  "keywords": ["dislike", "organization"]
}
```